In [1]:
import math, time, random, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("UNSW_NB15_training-set.csv")
test = pd.read_csv("UNSW_NB15_testing-set.csv")

In [3]:
train.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,0,600000000.0,0.0,0,0,0.006,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,0,850400000.0,0.0,0,0,0.010,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0


In [4]:
test.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0


In [5]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data = pd.concat([train,test]).reset_index(drop = True)
cols_cat = data.select_dtypes("object").columns
cols_numeric = data._get_numeric_data().columns

In [6]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data["service"].unique() # get rid of "-"
data["service"] = np.where(data["service"]=='-', "None", data["service"])
print(data["service"].unique())

['None' 'http' 'ftp' 'ftp-data' 'smtp' 'pop3' 'dns' 'snmp' 'ssl' 'dhcp'
 'irc' 'radius' 'ssh']


In [7]:
data = pd.get_dummies(data, columns = ["proto", "service", "state", "attack_cat"], dtype = "float64") 

In [8]:
data.drop(["attack_cat_Analysis", "attack_cat_Backdoor", "attack_cat_Exploits",
            "attack_cat_Fuzzers", "attack_cat_Reconnaissance", "attack_cat_Shellcode",
            "attack_cat_Worms", "attack_cat_Generic"], axis = 1, inplace = True)

In [9]:
data.head()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,...,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_None,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no,attack_cat_DoS,attack_cat_Normal
0,1,0.000011,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,0.000008,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,0.000005,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,0.000006,2,0,900,0,166666.6608,254,0,600000000.0,0.0,0,0,0.006,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0.000010,2,0,2126,0,100000.0025,254,0,850400000.0,0.0,0,0,0.010,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
data_drop = data.drop(["id", "attack_cat_DoS", "attack_cat_Normal"], axis = 1)

#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
cols_numeric = list(cols_numeric)
cols_numeric.remove("label")
cols_numeric.remove("id")

In [11]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data_drop[cols_numeric] = (data_drop[cols_numeric] - np.min(data_drop[cols_numeric])) / np.std(data_drop[cols_numeric])

In [12]:
data_drop.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,...,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_None,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
0,1.841222e-06,0.007356,0.0,0.002716,0.0,0.566962,2.478337,0.0,0.971102,0.0,0.0,0.0,1.589104e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.087903,0.0,0.0,0.0,0.092339,2.015524,0.000000,0.000000,0.0,0.089922,0.0,0.0,0.0,0.000000,0.091956,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.339070e-06,0.007356,0.0,0.010002,0.0,0.779572,2.478337,0.0,4.743422,0.0,0.0,0.0,1.155712e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.162200,0.0,0.0,0.0,0.092339,2.015524,0.000000,0.000000,0.0,0.089922,0.0,0.0,0.0,0.000000,0.091956,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8.369190e-07,0.007356,0.0,0.006008,0.0,1.247316,2.478337,0.0,4.600204,0.0,0.0,0.0,7.223200e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.476922,0.0,0.0,0.0,0.184678,2.015524,0.000000,0.000000,0.0,0.179844,0.0,0.0,0.0,0.000000,0.183913,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.004303e-06,0.007356,0.0,0.005041,0.0,1.039430,2.478337,0.0,3.230480,0.0,0.0,0.0,8.667840e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.068958,0.0,0.0,0.0,0.184678,2.015524,0.122343,0.122537,0.0,0.179844,0.0,0.0,0.0,0.119101,0.183913,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.673838e-06,0.007356,0.0,0.012096,0.0,0.623658,2.478337,0.0,4.578667,0.0,0.0,0.0,1.444640e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.046121,0.0,0.0,0.0,0.184678,2.015524,0.122343,0.122537,0.0,0.179844,0.0,0.0,0.0,0.119101,0.183913,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# now eda removals

In [14]:
data_drop.drop(["dur", "rate", "sloss", "djit", "tcprtt", "trans_depth", "ct_src_dport_ltm", "ct_flw_http_mthd",
                      "proto_3pc", "proto_aris", "proto_br-sat-mon", "proto_cphb", "proto_ddp", "proto_emcon",
                      "proto_ggp", "proto_iatp", "proto_idrp", "proto_ip", "proto_ipnip", "proto_ipv6-opts", 
                      "proto_iso-ip", "proto_leaf-1", "proto_micp", "proto_netblt", "proto_pim", "proto_ptp",
                      "proto_rsvp", "proto_sccopmce", "proto_sep", "proto_sprite-rpc", "proto_sun-nd", "proto_tp++",
                      "proto_unas", "proto_vrrp", "proto_xns-idp", "service_dns", "service_pop3", "service_ssl", "state_FIN",
                      "state_URN"], axis = 1, inplace = True)

In [15]:
data_drop.head()

,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,dloss,sinpkt,dinpkt,sjit,swin,stcpb,dtcpb,dwin,synack,ackdat,smean,dmean,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,...,proto_srp,proto_st2,proto_stp,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xtp,proto_zero,service_None,service_dhcp,service_ftp,service_ftp-data,service_http,service_irc,service_radius,service_smtp,service_snmp,service_ssh,state_ACC,state_CLO,state_CON,state_ECO,state_INT,state_PAR,state_REQ,state_RST,state_no
0,0.007356,0.0,0.002716,0.0,2.478337,0.0,0.971102,0.0,0.0,1.589104e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.087903,0.0,0.0,0.092339,2.015524,0.000000,0.0,0.089922,0.0,0.0,0.000000,0.091956,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.007356,0.0,0.010002,0.0,2.478337,0.0,4.743422,0.0,0.0,1.155712e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.162200,0.0,0.0,0.092339,2.015524,0.000000,0.0,0.089922,0.0,0.0,0.000000,0.091956,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.007356,0.0,0.006008,0.0,2.478337,0.0,4.600204,0.0,0.0,7.223200e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.476922,0.0,0.0,0.184678,2.015524,0.000000,0.0,0.179844,0.0,0.0,0.000000,0.183913,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.007356,0.0,0.005041,0.0,2.478337,0.0,3.230480,0.0,0.0,8.667840e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.068958,0.0,0.0,0.184678,2.015524,0.122343,0.0,0.179844,0.0,0.0,0.119101,0.183913,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.007356,0.0,0.012096,0.0,2.478337,0.0,4.578667,0.0,0.0,1.444640e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.046121,0.0,0.0,0.184678,2.015524,0.122343,0.0,0.179844,0.0,0.0,0.119101,0.183913,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn import model_selection
from sklearn import metrics
X = data_drop.drop("label", axis = 1)
Y = data_drop["label"]

In [17]:
global X

In [18]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
def fit_algo(algo, x, y, cv):
    model = algo.fit(x, y)
    
    #check its score
    acc = round(model.score(x, y) *100, 2)
    y_pred = model_selection.cross_val_predict(algo, x, y, cv = cv,
                                              n_jobs = -1)
    
    acc_cv = round(metrics.accuracy_score(Y,y_pred)*100,2)
    
    return y_pred, acc, acc_cv, model

In [19]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.linear_model import LogisticRegression
start_time = time.time()

pred_now, acc_lr, acc_cv_lr, lr = fit_algo(LogisticRegression(C = 0.1), X, Y, 10)

lr_time = (time.time() - start_time)

print("Accuracy: %s" % acc_lr)
print("Accuracy of CV: %s" % acc_cv_lr)
print("Execution time: %s" % lr_time)

Accuracy: 89.9
Accuracy of CV: 88.97
Execution time: 76.66110730171204


In [ ]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.neighbors import KNeighborsClassifier
start_time = time.time()
pred_now, acc_knn, acc_cv_knn, knn = fit_algo(KNeighborsClassifier(n_neighbors = 3)
                                        , X, Y, 10)
knn_time = (time.time() - start_time)
print("Accuracy: %s" % acc_knn)
print("Accuracy of CV: %s" % acc_cv_knn)
print("Execution time: %s" % knn_time)

In [ ]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
start_time = time.time()
pred_now, acc_dt, acc_cv_dt, dt = fit_algo(DecisionTreeClassifier(random_state = 1)
                                        , X, Y, 10)

dt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_dt)
print("Accuracy of CV: %s" % acc_cv_dt)
print("Execution time: %s" % dt_time)




In [ ]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Random Forest
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()
pred_now, acc_rf, acc_cv_rf, rf = fit_algo(RandomForestClassifier(n_estimators = 100)
                                        , X, Y, 10)

rf_time = (time.time() - start_time)

print("Accuracy: %s" % acc_rf)
print("Accuracy of CV: %s" % acc_cv_rf)
print("Execution time: %s" % rf_time)

In [ ]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gaussian Naive Bayes 
from sklearn.naive_bayes import GaussianNB
start_time = time.time()

pred_now, acc_gnb, acc_cv_gnb, gnb= fit_algo(GaussianNB()
                                        ,X,Y,5)

gnb_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gnb)
print("Accuracy of CV: %s" % acc_cv_gnb)
print("Execution time: %s" % gnb_time)

In [ ]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gradient Boosting


from sklearn.ensemble import GradientBoostingClassifier
start_time = time.time()

pred_now, acc_gbt, acc_cv_gbt, gbt= fit_algo(GradientBoostingClassifier()
                                        , X, Y, 10)

gbt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gbt)
print("Accuracy of CV: %s" % acc_cv_gbt)
print("Execution time: %s" % gbt_time)



In [ ]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Cross Validation Accuracy Comparison
algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'CV Accuracy %' : [acc_cv_lr, acc_cv_dt, acc_cv_rf, acc_cv_gnb, acc_cv_gbt, acc_cv_knn] })
acc_df = acc_df.sort_values(by='CV Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df

In [ ]:

# Accuracy Comparison

algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'Accuracy %' : [acc_lr, acc_dt, acc_rf, acc_gnb, acc_gbt, acc_knn] })
acc_df = acc_df.sort_values(by='Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df

In [ ]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
fimp_rf = pd.DataFrame({'Feature' : X.columns, 'Importance' : (rf.feature_importances_).astype(float)})
fimp_rf = fimp_rf.sort_values(by='Importance', ascending=False)
fimp_rf.head(37)

50%

In [20]:
x = data_drop.copy()

In [21]:
from sklearn.model_selection import train_test_split
SEED = 1

x_train, x_test = train_test_split(x, train_size = 0.50, random_state = SEED)

In [22]:
print(x_train.shape)
x_train.head()

(128836, 157)


,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,dloss,sinpkt,dinpkt,sjit,swin,stcpb,dtcpb,dwin,synack,ackdat,smean,dmean,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,...,proto_srp,proto_st2,proto_stp,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xtp,proto_zero,service_None,service_dhcp,service_ftp,service_ftp-data,service_http,service_irc,service_radius,service_smtp,service_snmp,service_ssh,state_ACC,state_CLO,state_CON,state_ECO,state_INT,state_PAR,state_REQ,state_RST,state_no
115986,0.007356,0.017859,0.000702,0.001218,0.302474,0.257179,0.003157,0.296331,0.0,1.444640e-06,0.000005,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.237979,0.350338,0.0,0.000000,0.000000,0.122343,0.000000,0.089922,0.0,0.0,0.476403,0.183913,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
201332,0.007356,0.000000,0.000518,0.000000,2.478337,0.000000,0.613791,0.000000,0.0,5.778560e-07,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.160271,0.000000,0.0,1.385082,2.015524,1.835147,2.572235,1.348832,0.0,0.0,1.905613,1.379344,0.000000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
192002,0.007356,0.000000,0.001013,0.000000,2.478337,0.000000,0.478590,0.000000,0.0,1.300176e-06,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.369110,0.000000,0.0,0.369355,2.015524,0.244686,0.342965,0.359689,0.0,0.0,0.238202,0.367825,0.000000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
80789,0.007356,0.000000,0.000380,0.000000,2.478337,0.000000,0.193829,0.000000,0.0,1.444640e-06,0.000000,0.0,2.002085,0.0,0.0,0.0,0.0,0.0,0.101991,0.000000,0.0,0.646372,6.046572,0.244686,0.000000,0.539533,0.0,0.0,0.476403,0.551738,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
79769,0.000000,0.000000,0.000127,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,8.667941e+00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.106848,0.000000,0.0,0.000000,2.015524,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,8.430444,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [23]:
from sklearn import model_selection
from sklearn import metrics
X = x_train.drop("label", axis = 1)
Y = x_train["label"]

In [24]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.linear_model import LogisticRegression
start_time = time.time()

pred_now, acc_lr, acc_cv_lr, lr = fit_algo(LogisticRegression(C = 0.1), X, Y, 10)

lr_time = (time.time() - start_time)

print("Accuracy: %s" % acc_lr)
print("Accuracy of CV: %s" % acc_cv_lr)
print("Execution time: %s" % lr_time)

Accuracy: 89.89
Accuracy of CV: 89.84
Execution time: 37.959532260894775


In [25]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.neighbors import KNeighborsClassifier
start_time = time.time()
pred_now, acc_knn, acc_cv_knn, knn = fit_algo(KNeighborsClassifier(n_neighbors = 3)
                                        , X, Y, 10)
knn_time = (time.time() - start_time)
print("Accuracy: %s" % acc_knn)
print("Accuracy of CV: %s" % acc_cv_knn)
print("Execution time: %s" % knn_time)

Accuracy: 95.62
Accuracy of CV: 91.17
Execution time: 950.9041483402252


In [26]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Decision Tree


from sklearn.tree import DecisionTreeClassifier
start_time = time.time()
pred_now, acc_dt, acc_cv_dt, dt = fit_algo(DecisionTreeClassifier(random_state = 1)
                                        , X, Y, 10)

dt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_dt)
print("Accuracy of CV: %s" % acc_cv_dt)
print("Execution time: %s" % dt_time)




Accuracy: 99.8
Accuracy of CV: 93.25
Execution time: 19.415203094482422


In [27]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Random Forest
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()
pred_now, acc_rf, acc_cv_rf, rf = fit_algo(RandomForestClassifier(n_estimators = 100)
                                        , X, Y, 10)

rf_time = (time.time() - start_time)

print("Accuracy: %s" % acc_rf)
print("Accuracy of CV: %s" % acc_cv_rf)
print("Execution time: %s" % rf_time)

Accuracy: 99.8
Accuracy of CV: 94.71
Execution time: 172.2976884841919


In [28]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gaussian Naive Bayes 
from sklearn.naive_bayes import GaussianNB
start_time = time.time()

pred_now, acc_gnb, acc_cv_gnb, gnb= fit_algo(GaussianNB()
                                        ,X,Y,5)

gnb_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gnb)
print("Accuracy of CV: %s" % acc_cv_gnb)
print("Execution time: %s" % gnb_time)

Accuracy: 42.17
Accuracy of CV: 42.17
Execution time: 2.289635419845581


In [29]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
start_time = time.time()

pred_now, acc_gbt, acc_cv_gbt, gbt= fit_algo(GradientBoostingClassifier()
                                        , X, Y, 10)

gbt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gbt)
print("Accuracy of CV: %s" % acc_cv_gbt)
print("Execution time: %s" % gbt_time)



Accuracy: 93.43
Accuracy of CV: 93.33
Execution time: 600.2451207637787


In [30]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Cross Validation Accuracy Comparison


algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'CV Accuracy %' : [acc_cv_lr, acc_cv_dt, acc_cv_rf, acc_cv_gnb, acc_cv_gbt, acc_cv_knn] })
acc_df = acc_df.sort_values(by='CV Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df


,Algorithm,CV Accuracy %
0,RandomForest,94.71
1,GBC,93.33
2,Decision Tree,93.25
3,KNN,91.17
4,Log. Reg.,89.84
5,Gaussian NB,42.17


In [31]:
# Accuracy Comparison

algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'Accuracy %' : [acc_lr, acc_dt, acc_rf, acc_gnb, acc_gbt, acc_knn] })
acc_df = acc_df.sort_values(by='Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df

,Algorithm,Accuracy %
0,Decision Tree,99.80
1,RandomForest,99.80
2,KNN,95.62
3,GBC,93.43
4,Log. Reg.,89.89
5,Gaussian NB,42.17


In [32]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda

fimp_rf = pd.DataFrame({'Feature' : X.columns, 'Importance' : (rf.feature_importances_).astype(float)})
fimp_rf = fimp_rf.sort_values(by='Importance', ascending=False)
fimp_rf.head(37)



,Feature,Importance
4,sttl,0.102895
22,ct_state_ttl,0.084435
2,sbytes,0.059444
6,sload,0.049951
5,dttl,0.048940
7,dload,0.048068
18,smean,0.042490
3,dbytes,0.038187
29,ct_srv_dst,0.038122
11,sjit,0.035720
